# import libraries

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# define functions

In [44]:
####################################
#
#
#   define functions to solve equilibrium
#
#
#####################################

def solve_t(N_t, p):     
    """
    solves single period of the malthus model given population N.
    ----------
    parameters
    ----------
    N_t  : float
         current population.
    p    : {flt x 6}
         parameter dictionary: N_0, X, A, θ, α, T
    -------
    returns
    -------
    {flt x 6}, flt
        agg labor, output, consumption, ...
        ...and per-capita output, consumption, land...
        ... and agg population next period.
    """
    # unpack parameters for clarify
    N_0, X, A, θ, α, T = p.values()
    
    # initialize period solutions
    v_t = {}
    
    v_t['x'] = X / N_t            # land per capita
    v_t['y'] = A * v_t['x']**θ    # output per capita
    v_t['c'] = v_t['y']           # consupmtion per capita
    v_t['L'] = N_t                # land
    v_t['Y'] = v_t['y'] * N_t     # output
    v_t['C'] = v_t['c'] * N_t     # consumption
    NN = v_t['c']**α * N_t        # population next period

    return v_t, NN
    
    
    
def solve_path(p):
    """
    solves equilibrium path of the malthus model.
    ----------
    parameters
    ----------
    p    : {flt x 6}
         parameter dictionary: N_0, X, A, θ, α, T
    -------
    returns
    -------
    {np.array(float: T x 1) x 7}
        agg population, labor, output, consumption, ...
        ...and per-capita output, consumption, and land.
    """
    
    # unpack parameters for clarify
    N_0, X, A, θ, α, T = p.values()    

    # initialize equilibrium paths
    v = {i: np.zeros(T + 1) for i in ['N', 'L', 'Y', 'C', 'y', 'c', 'x']}

    # solve for model variables
    v['N'][0] = N_0
    for t in range(T):
        
        # solve for period outcomes
        v_t, v['N'][t + 1] = solve_t(v['N'][t], p)
        
        # add period solutions to path 
        for key, val in v_t.items():
            
            v[key][t] = val
        
    return v



def solve_ss(p):
    """
    solves steady state of the malthus model.
    ----------
    parameters
    ----------
    p    : {flt x 6}
         parameter dictionary: N_0, X, A, θ, α, T
    -------
    returns
    -------
    {flt x 7}
        agg population, output, consumption ...
        ... and per-capita output, consumption, and land.
    """
    
    # unpack parameters for clarify
    N_0, X, A, θ, α, T = p.values()    
    
    # initialize steady state solution
    v_ss = {}
    
    # solve for steady state 
    v_ss['c'] = 1**(1 / α)             # consumption per capita
    v_ss['y'] = v_ss['c']              # output per capita
    v_ss['x'] = A**(-1 / θ)            # land per capita
    v_ss['N'] = X / v_ss['x']          # population
    v_ss['L'] = v_ss['N']              # labor
    v_ss['Y'] = v_ss['y'] * v_ss['N']  # output
    v_ss['C'] = v_ss['c'] * v_ss['N']  # consumption
    
    return v_ss



####################################
#
#
#   define functions to plot outcomes
#
#
#####################################

def plot_variable(lbl, var1, var2, var1ss, var2ss):
    """
    plots two sets of model solutions for a given variable
    ----------
    parameters
    ----------
    lbl     : str
            variable label
    var1    : np.array(T)
            variable path given parameters p1
    var2    : np.array(T)
            variable path given parameters p2  
    var1ss  : flt
            variable ss value given parameters p1
    var2ss  : flt
            variable ss value given parameters p2         
    -------
    returns
    -------
    None.
    """
    
    # initialize figure
    fig, ax = plt.subplots()
    
    # plot paths
    ax.plot(var1, marker='o', markevery=5, label='p1', color='tab:blue')
    ax.plot(var2, marker='^', markevery=5, label='p2', color='tab:orange')
    
    # plot steady state values
    ax.axhline(y=var1ss, ls='--', lw=0.5, color='tab:blue')
    ax.axhline(y=var2ss, ls='--', lw=0.5, color='tab:orange')
    
    # finish figure
    ax.set_ylim(bottom=0)
    ax.set_xlabel('Time, T')
    ax.set_ylabel(lbl)
    ax.legend(frameon=False)
    plt.show()
    
    

def plot_eq(p1, p2):
    """
    plots model solutions for all variables given two parameter sets p1 and p2
    ----------
    parameters
    ----------
    p1  : {flt x 6}
        parameter dictionary: N_0, X, A, θ, α, T
    p2  : {flt x 6}
        parameter dictionary: N_0, X, A, θ, α, T         
    -------
    returns
    -------
    {np.array(float: T x 1) x 7}, 
    {np.array(float: T x 1) x 7}, 
    {flt x 7}, {flt x 7}
        agg population, output, consumption ...
        ... and per-capita output, consumption, and land ...
        equilibrium path, then steady state values ...
        alternately for parameters p1 and p2.
    """
    
    # solve for paths
    v1 = solve_path(p1)
    v2 = solve_path(p2)
    
    # solve for steady states
    v1_ss = solve_ss(p1)
    v2_ss = solve_ss(p2)
    
    # iterate through all variables
    for i in v1.keys():
        
        # plot outcomes for variable
        plot_variable(i, v1[i][:-1], v2[i][:-1], v1_ss[i], v2_ss[i])
    
    return v1, v2, v1_ss, v2_ss

# set parameters and solve for equilibria

In [ ]:
####################################
#
#
#   set parameters
#
#
#####################################

# computational parameters
T = 50    # number of periods to solve for

# model parameters: I
p1 = {'N_0' : 1,    # initial population
      'X' : 100,    # total land endowment
      'A' : 1,      # total factor productivity
      'θ' : 0.33,   # land's weight in production function
      'α' : 0.5,    # elasticity of population growth wrt consumption per capita
      'T' : T       # number of periods to solve for  
     }

# model parameters: II
p2 = p1.copy()
p2['N_0'] = 2



####################################
#
#
#   solve for equilibria and plot
#
#
#####################################

v1, v2, v1_ss, v2_ss = plot_eq(p1, p2)